In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import sys
sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

from torchmetrics.metric import Metric
import torch
import torchvision
from torchvision import transforms
import albumentations as A
from albumentations.augmentations.geometric.transforms import Affine
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import Tensor, nn
! pip install torchsummary
! pip install torchcontrib
from efficientnet_pytorch import EfficientNet
from torchsummary import summary
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchcontrib.optim import SWA
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df

# Cài đặt tham số

In [ ]:
class CFG:
    # data path
    train_csv_path = '../input/plant-pathology-2021-fgvc8/train.csv'
#     train_imgs_dir = '../input/resized-plant2021/img_sz_256'
    train_imgs_dir = '../input/resized-plant2021/img_sz_512'
    # label info
    label_num2str = {0: 'powdery_mildew',
                     1: 'scab',
                     2: 'complex',
                     3: 'frog_eye_leaf_spot',
                     4: 'rust'}
    
    label_str2num = {'powdery_mildew': 0,
                     'scab': 1,
                     'complex': 2,
                     'frog_eye_leaf_spot': 3,
                     'rust': 4}
    # model info
    model_name = 'pp-eff-n3'
    pretrained_backbone = 'efficientnet-b7'
    # training hyper-parameters
    fl_alpha = 1.0  # alpha of focal_loss
    fl_gamma = 2.0  # gamma of focal_loss
    cls_weight = [3.6480, 1.0001, 2.1840, 1.5001, 2.2901] # class weights for we calculating loss
    use_swa = True
    seed = 77
    num_classes = 5
    num_epochs = 5
    batch_size = 16
    t_max = 18
    lr = 1e-3
    min_lr = 1e-6
    n_fold = 6
    num_workers = 8
    accum_grad_batch = 1
    early_stop_delta = 1e-7
    # if you have multi-GPU on your machine, set gpu_list to [0, 1, ...]
    gpu_idx = 0
    device = torch.device(f'cuda:{gpu_idx}' if torch.cuda.is_available() else 'cpu')
    gpu_list = [gpu_idx]

In [ ]:
def encode_label():
    label_split = []
    for label in df['labels']:
        list_single_label = label.split(' ')
        label_enc = np.array([0, 0, 0, 0, 0])
        for single_label in list_single_label:
            if single_label != 'healthy':
                label_enc[CFG.label_str2num.get(single_label)] = 1
        label_split.append(label_enc)
    return np.array(label_split)

encoded_label = Tensor(encode_label())

# Augmentation cho ảnh

In [ ]:
SIZE=224
DATASET_IMAGE_MEAN = (0.485, 0.456, 0.406)
DATASET_IMAGE_STD = (0.229, 0.224, 0.225)


train_transform = Compose([
#     A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    Affine(
        scale=(0.8, 1.0), 
        translate_percent=(0.1, 0.3), 
        rotate=(-360, 360),
        cval=0, cval_mask=255, fit_output=True, 
        always_apply=False, p=0.8
    ),
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(mean=DATASET_IMAGE_MEAN, std=DATASET_IMAGE_STD, p=1.0),
    ToTensorV2()
])

valid_transform = Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(mean=DATASET_IMAGE_MEAN, std=DATASET_IMAGE_STD, p=1.0),
    ToTensorV2()
])

# Xây dựng Dataset và DataLoader

In [ ]:
class PlantDataset(Dataset):
    def __init__(self, root_dir, file_list, labels, transform):
        self.file_list = file_list
        self.labels = labels
        self.transform = transform
        self.root_dir = root_dir
   
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = np.array(Image.open(os.path.join(self.root_dir, img_path)))

        img_transformed = self.transform(image=img)['image']

        return img_transformed, self.labels[index]

In [ ]:
# TRAIN_IMG_ROOT = '../input/resized-plant2021/img_sz_384'

# train_dataset = PlantDataset(
#     root_dir=CFG.train_imgs_dir, 
#     file_list=df['image'][idx_train].to_numpy(), labels=y_train, 
#     transform=train_transform
# )

# valid_dataset = PlantDataset(
#     root_dir=CFG.train_imgs_dir, 
#     file_list=df['image'][idx_valid].to_numpy(), labels=y_valid, 
#     transform=valid_transform
# )

# train_dataloader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
# valid_dataloader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False)

In [ ]:
# fig, axs = plt.subplots(nrows=8, ncols=4, dpi=100, figsize=(18, 20))

# for i, (inputs, targets) in enumerate(train_dataloader):
#     [axs[i][j].imshow(inputs[i * 4 + j].permute(1, 2, 0)) for i in range(8) for j in range(4)]
#     break

# Xây dựng Model

In [ ]:
"""
Xây dựng class FocalLoss phục vụ cho training
"""

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from Nan-loss error
        self.cls_weights = torch.tensor([CFG.cls_weight],dtype=torch.float, requires_grad=False, device=CFG.device)

    def forward(self, logits, target):
        """
        logits & target có dạng [batch_size, num_classes]
        """
        probs = torch.sigmoid(logits)
        one_subtract_probs = 1.0 - probs
        # add epsilon
        probs_new = probs + self.epsilon
        one_subtract_probs_new = one_subtract_probs + self.epsilon
        # calculate focal loss
        log_pt = target * torch.log(probs_new) + (1.0 - target) * torch.log(one_subtract_probs_new)
        pt = torch.exp(log_pt)
        focal_loss = -1.0 * (self.alpha * (1 - pt) ** self.gamma) * log_pt
        focal_loss = focal_loss * self.cls_weights
        return torch.mean(focal_loss)

In [ ]:
"""
Định nghĩa cách tính điểm F1-Score 
"""
class MyF1Score(Metric):
    def __init__(self, cfg, threshold: float = 0.5, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.cfg = cfg
        self.threshold = threshold
        self.add_state("tp", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("fp", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("fn", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        preds_str_batch = self.num_to_str(torch.sigmoid(preds))
        target_str_batch = self.num_to_str(target)
        tp, fp, fn = 0, 0, 0
        for pred_str_list, target_str_list in zip(preds_str_batch, target_str_batch):
            for pred_str in pred_str_list:
                if pred_str in target_str_list:
                    tp += 1
                if pred_str not in target_str_list:
                    fp += 1

            for target_str in target_str_list:
                if target_str not in pred_str_list:
                    fn += 1
        self.tp += tp
        self.fp += fp
        self.fn += fn

    def compute(self):
        f1 = 2.0 * self.tp / (2.0 * self.tp + self.fn + self.fp)
        return f1
    
    def num_to_str(self, ts: torch.Tensor) -> list:
        batch_bool_list = (ts > self.threshold).detach().cpu().numpy().tolist()
        batch_str_list = []
        for one_sample_bool in batch_bool_list:
            lb_str_list = [self.cfg.label_num2str[lb_idx] for lb_idx, bool_val in enumerate(one_sample_bool) if bool_val]
            if len(lb_str_list) == 0:
                lb_str_list = ['healthy']
            batch_str_list.append(lb_str_list)
        return batch_str_list

In [ ]:
# for name, param in model.named_parameters():
#     print(name)
#     param.requires_grad = False
# model(torch.randn(10, 3, 240, 240)).shape
# summary(model, (3, 240, 240))

In [ ]:
"""
Định nghĩa mô hình
"""

class MyNetwork(pl.LightningModule):
    def __init__(self, cfg):
        super(MyNetwork, self).__init__()
        self.cfg = cfg
        self.model = EfficientNet.from_pretrained(cfg.pretrained_backbone, num_classes=CFG.num_classes)
        self.criterion = FocalLoss(cfg.fl_alpha, cfg.fl_gamma)
        self.metric = MyF1Score(cfg)
       
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        if self.cfg.use_swa:
            self.optimizer = SWA(torch.optim.Adam(self.model.parameters(), lr=self.cfg.lr))
        else:
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.cfg.lr)
            
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer,
                                                                    T_max=self.cfg.t_max,
                                                                    eta_min=self.cfg.min_lr,
                                                                    verbose=True)
        return {'optimizer': self.optimizer, 'lr_scheduler': self.scheduler}
    
    def training_step(self, batch, batch_idx):
        img_ts, lb_ts = batch
        pred_ts = self.model(img_ts)
        loss = self.criterion(pred_ts, lb_ts)
        score = self.metric(pred_ts, lb_ts)
        logs = {'train_loss': loss, 'train_f1': score, 'lr': self.optimizer.param_groups[0]['lr']}
        self.log_dict(logs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        img_ts, lb_ts = batch
        pred_ts = self.model(img_ts)
        loss = self.criterion(pred_ts, lb_ts)
        score = self.metric(pred_ts, lb_ts)
        logs = {'valid_loss': loss, 'valid_f1': score}
        self.log_dict(logs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

# Tối ưu tham số của mô hình

In [ ]:
def start_train(fold_idx, idx_train, idx_valid):
    """
    Khởi tạo Trainer và Logger
    """
    logger = CSVLogger(save_dir=f'fold{fold_idx}_logs/', name=CFG.model_name)
    #     logger.log_hyperparams(CFG.__dict__)
    checkpoint_callback = ModelCheckpoint(monitor='valid_f1',
                                          save_top_k=1,
                                          save_last=True,
                                          save_weights_only=True,
                                          filename='best_perform',
                                          verbose=False,
                                          mode='max')

    trainer = Trainer(max_epochs=CFG.num_epochs,
                      gpus=1,
                      accumulate_grad_batches=CFG.accum_grad_batch,
                      callbacks=[checkpoint_callback],
                      logger=logger,
                      weights_summary='top')
    """
    Khởi tạo Dataset và DataLoader
    """
    train_dataset = PlantDataset(
        root_dir=CFG.train_imgs_dir,
        file_list=df['image'][idx_train].to_numpy(), labels=encoded_label[idx_train], 
        transform=train_transform
    )

    valid_dataset = PlantDataset(
        root_dir=CFG.train_imgs_dir, 
        file_list=df['image'][idx_valid].to_numpy(), labels=encoded_label[idx_valid], 
        transform=valid_transform
    )

    train_dataloader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False)

    """
    Khởi tạo Model
    """
    model = MyNetwork(CFG)

    """
    Huấn luyện mô hình
    """
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

In [ ]:
k_fold = KFold(n_splits=CFG.n_fold, shuffle=True)
idx = [i for i in range(len(df.index))]

for fold_idx, (idx_train, idx_valid) in enumerate(k_fold.split(idx)):
    start_train(fold_idx, idx_train, idx_valid)

In [ ]:
# CFG.n_fold = 1
# idx = [i for i in range(len(df.index))]
# idx_train, idx_valid, y_train, y_valid = train_test_split(idx, encoded_label)
# start_train(0, idx_train, idx_valid)

In [ ]:
"""
Vẽ biểu đồ độ thay đổi của f1-score và hàm loss
"""


fig = plt.figure(figsize=(32, 10), constrained_layout=True)
gs = gridspec.GridSpec(2, CFG.n_fold, figure=fig)


for fold_idx in range(CFG.n_fold):
    tmp_log_dir = f"fold{fold_idx}_logs/{CFG.model_name}/version_0"
    metrics = pd.read_csv(os.path.join(tmp_log_dir, 'metrics.csv'))

    train_acc = metrics['train_f1'].dropna().reset_index(drop=True)
    valid_acc = metrics['valid_f1'].dropna().reset_index(drop=True)
    
    ax = fig.add_subplot(gs[0, fold_idx])
    ax.plot(train_acc, color="r", marker="o", label='train/f1')
    ax.plot(valid_acc, color="b", marker="x", label='valid/f1')
    ax.set_xlabel('Epoch', fontsize=24)
    ax.set_ylabel('F1', fontsize=24)
    ax.set_title(f'fold {fold_idx}')
    ax.legend(loc='lower right', fontsize=18)


    train_loss = metrics['train_loss'].dropna().reset_index(drop=True)
    valid_loss = metrics['valid_loss'].dropna().reset_index(drop=True)

    ax = fig.add_subplot(gs[1, fold_idx])
    ax.plot(train_loss, color="r", marker="o", label='train/loss')
    ax.plot(valid_loss, color="b", marker="x", label='valid/loss')
    ax.set_ylabel('Loss', fontsize=24)
    ax.set_xlabel('Epoch', fontsize=24)
    ax.legend(loc='upper right', fontsize=18)

In [ ]:
# ! ls ./fold0_logs/pp-eff-n3/version_0/checkpoints